# このJupyter Notebookの使い方
これはレシピ集として作成されています。最初に、初期設定セクションにて利用するElasticsearch環境やOpenAI環境、HuggingFace環境の接続情報を設定します。その後は、実行したいセクションA. B. C. ..から始めて順番にコマンドを実行してください。
そのためにセクション間で重複したコードが繰り返しあります。

# 1. 初期設定

## ライブラリの有効化

In [177]:
from pprint import pprint
import os
from getpass import getpass

## Elasticsearchの設定

In [178]:
from elasticsearch import Elasticsearch

ELASTIC_API_KEY = getpass("Elastic deployment API Key")
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_URL = getpass("Elastic deployment URL. No need if Cloud ID is provided.")
ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")

if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    # request_timeout=300,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

ObjectApiResponse({'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## OpenAI APIの設定

In [179]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

## HuggingFace APIの設定

In [180]:
huggingface_api_Key = getpass("Enter your HF Inference API Key:\n\n")

## Elastic 上のMLモデルの設定

In [181]:
%pip install -q eland elasticsearch transformers sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [182]:
import tempfile
import os
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel

In [183]:
def load_model(es_connection, model_id, task_type):
  with tempfile.TemporaryDirectory() as tmp_dir:
    print(f"Loading HuggingFace transformer tokenizer and model [{model_id}] for task [{task_type}]" )

    tm = TransformerModel(model_id=model_id, task_type=task_type)
    model_path, config, vocab_path = tm.save(tmp_dir)

    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    model_exists = es_connection.options(ignore_status=404).ml.get_trained_models(model_id=ptm.model_id).meta.status == 200

    if model_exists:
      print("Model has already been imported")
    else:
      print("Importing model")
      ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)
      print(f"Model successfully imported with id '{ptm.model_id}'")

def start_model(es_connection, model_id, task_type):
    tm = TransformerModel(model_id=model_id, task_type=task_type)
    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    print("Starting model deployment")
    ptm.start()
    print(f"Model successfully started with id '{ptm.model_id}'")

def stop_model(es_connection, model_id, task_type):
    tm = TransformerModel(model_id=model_id, task_type=task_type)
    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    ptm.stop()


### Elastic ML E5モデルのロード

In [184]:
es.info()
load_model(es, "intfloat/multilingual-e5-base", "text_embedding")

Loading HuggingFace transformer tokenizer and model [intfloat/multilingual-e5-base] for task [text_embedding]
Model has already been imported


### Elastic ML E5モデルの開始

In [187]:
print(es.info())
start_model(es, "intfloat/multilingual-e5-base", "text_embedding")

{'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Starting model deployment
Model successfully started with id 'intfloat__multilingual-e5-base'


### (Elastic ML E5モデルの停止)

In [186]:
es.info()
stop_model(es, "intfloat/multilingual-e5-base", "text_embedding")

/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/eland/ml/pytorch/_pytorch_model.py:152: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  self._client.ml.stop_trained_model_deployment(model_id=self.model_id)


# 2. 検索対象のドキュメントをダウンロードして整形

In [188]:
%pip install -qU pytest-playwright beautifulsoup4
!playwright install

Note: you may need to restart the kernel to use updated packages.
zsh:1: /Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/bin/playwright: bad interpreter: /Users/nobuhikosekiya/lab/projects/es-langchain/.venv/bin/python: no such file or directory


## 2-A. WebBaseLoaderでWebページのテキストを抽出したDocumentを作る
このセクションでは、指定のWebページに含まれる全テキストを単純抽出して、Documentとして格納します。
結果は**docs**のリストに格納されます。

In [14]:
from langchain.document_loaders import WebBaseLoader
from pprint import pprint

urls = [
    "https://www.elastic.co/jp/what-is/root-cause-analysis",
    "https://www.elastic.co/jp/what-is/anomaly-detection",
    "https://www.elastic.co/jp/what-is/siem",
    "https://www.elastic.co/jp/what-is/opentelemetry",
    "https://www.elastic.co/jp/what-is/aiops",
    "https://www.elastic.co/jp/what-is/kubernetes-monitoring"
    ]

loader = WebBaseLoader(urls)
docs = loader.load()
pprint(docs, indent=4, width=30)

[   Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニアリングのスピードを高め、イノベー

## 2-B. ページ内の不要な情報を削ぎ落としたDocumentを作成する
ここでは、以下の方法でメインのコンテンツだけを抽出するようにしています。
1. メインのコンテンツが含まれるHTMLのmainタグを抽出します。
2. 不要なscriptタグを除去しています。
3. さらにmainの中の必要なタグ h1, h3, h4, p, ulなどに絞ってテキストを抽出します。

結果のWebページのテキストが含まれたDocumentは**docs**のリストに格納されます。

In [353]:
from langchain.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from pprint import pprint

def extract_main_content(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    main_html = soup.find("main")
    scripts = main_html.find_all('script')
    for s in scripts:
        s.extract()

    if main_html:
        return main_html
    return None

urls = [
    "https://www.elastic.co/jp/what-is/root-cause-analysis",
    "https://www.elastic.co/jp/what-is/anomaly-detection",
    "https://www.elastic.co/jp/what-is/siem",
    "https://www.elastic.co/jp/what-is/opentelemetry",
    "https://www.elastic.co/jp/what-is/aiops",
    "https://www.elastic.co/jp/what-is/kubernetes-monitoring"
    ]
loader = AsyncHtmlLoader(urls) # Use to load raw html into page_content
html_docs = loader.load()

for html_doc in html_docs:
#     pprint(html_doc.metadata)
    # print("======= 抽出前 =======")
    # pprint(html_doc.page_content[:1000])
    # print("======= 抽出後 =======")
    extraced_html = extract_main_content(f"<html><body>{html_doc.page_content}</body></html>")
    html_doc.page_content = f"<html><body>{extraced_html}</body></html>"
    # pprint(html_doc.page_content[:1000])
    # print("==============")

Fetching pages: 100%|##########| 6/6 [00:00<00:00,  9.07it/s]


In [352]:
from langchain.document_transformers import BeautifulSoupTransformer
bs_transformer = BeautifulSoupTransformer()

for html_doc in html_docs:
    extracted_text = bs_transformer.extract_tags(html_content=html_docs[0].page_content, tags=["h1","h3","h4","p","ul"])
    # pprint(extracted_text)
    html_doc.page_content = extracted_text
# docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["a"])
# docs_transformed

docs = html_docs

# 3. 様々なサーチを試そう

In [176]:
# 結果を入れて後で見るための箱
result_list = []

## 3-A. Elasticsearchのキーワード検索 (BM25)

### インデックス作成
Kuromojiをアナライザーとして設定したElasticsearchのインデックスを作成します。

In [5]:
%pip install -q elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [6]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from getpass import getpass

In [163]:
INDEX_NAME="test_index_bm25"

In [164]:

if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)


Index 'test_index_bm25' does not exist.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index_bm25'})

In [165]:
es.indices.close(index=INDEX_NAME)

add_settings = {
  "index": {
    "analysis": {
      "char_filter": {
        "normalize": {
          "mode": "compose",
          "name": "nfkc",
          "type": "icu_normalizer"
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_index_synonym": {
          "type": "synonym",
          "lenient": "false",
          "synonyms": []
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "tokenizer": {
        "ja_kuromoji_tokenizer": {
          "mode": "search",
          "discard_compound_token": "true",
          "type": "kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

# Define a synonym set
synonyms_set = [
    # {
    #   "id": "test-1",
    #   "synonyms": "foo, bar"
    # },
    # {
    #   "id": "test-2",
    #   "synonyms": "test => check"
    # }
  ]

# Define the identifier for the synonym set
synonym_id = "ja_search_synonym-set"

from elasticsearch.client import SynonymsClient
# Call the put_synonym function to add or update the synonym set
response = SynonymsClient(es).put_synonym(id=synonym_id, synonyms_set=synonyms_set)

# Define the new settings you want to apply
add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_search_synonym": {
          "type": "synonym_graph",
          "synonyms_set": "ja_search_synonym-set",
          "updateable": "true"
        }
      },
      "analyzer": {
        "ja_kuromoji_index_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_index_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        },
        "ja_kuromoji_search_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_search_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

es.indices.open(index=INDEX_NAME, request_timeout=60)


/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/1678612302.py:119: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [166]:
es.indices.close(index=INDEX_NAME, request_timeout=60)

# Define the new mapping
add_mapping = {
  "properties": {
    "page_content": {
      "type": "text",
      "fields": {
        "kuromoji": {
          "type": "text",
          "analyzer": "ja_kuromoji_index_analyzer",
          "search_analyzer": "ja_kuromoji_search_analyzer"
        }
      }
    },
    "metadata": {
      "type": "object",
      "properties": {
        "source": {
          "type": "keyword"
        }
      }
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/2817861892.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### インジェスト

In [167]:
from elasticsearch import Elasticsearch, helpers

docs_json = [doc.to_json()["kwargs"] for doc in docs]
# print(docs_json)

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs)

response = es.search(index=INDEX_NAME, query={"query": {"match_all": {}}}, fields=["metadata"])
for hit in response['hits']['hits']:
    metadata = hit['_source']['metadata']
    print(f"metadata: {metadata}")

(6, [])

### サーチ

In [168]:
def do_search(query_text):
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query_text,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    fields = ["metadata"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields)

    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        score = hit['_score']
        print(f"_score: {score}, metadata: {metadata}")

    return {"query": query_text, "result": [ hit['_source']['metadata'] for hit in response['hits']['hits'] ]}

In [169]:
query = "異常検知について知りたい"
result = do_search(query)
result_list.append({"search_logic": "bm25", "query": query, "result": result})

_score: 2.085009, metadata: {'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
_score: 1.4552612, metadata: {'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
_score: 0.91729116, metadata: {'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
_score: 0.86592853, metadata: {'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプ

In [46]:
query = "機械学習の機能ついて知りたい"
result = do_search(query)
result_list.append({"search_logic": "bm25", "query": query, "result": result})

_score: 1.78886, metadata: {'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
_score: 1.486545, metadata: {'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
_score: 0.44918907, metadata: {'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
_score: 0.4148178, metadata: {'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': '

In [174]:
query = "システム監視においての高度な分析方法が知りたい"
result = do_search(query)
result_list.append({"search_logic": "bm25", "query": query, "result": result})

_score: 2.429951, metadata: {'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
_score: 2.2458336, metadata: {'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
_score: 1.8270609, metadata: {'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
_score: 1.6567736, metadata: {'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するための

## 3-B. （Open AI Embeddingを利用） Elasticsearchのセマンティック検索
LangchainのElasticsearchStoreというクラスを使って色々なAI/MLモデルを切り替えながらElasticsearchのベクトル検索をテストしていくことができます。

参考：https://python.langchain.com/docs/integrations/vectorstores/elasticsearch

In [ ]:
%pip install -q elasticsearch langchain openai tiktoken

1189.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [25]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from getpass import getpass

### インジェスト
補足：ElasticsearchStoreの中で自動的にインデックスの作成も行われています。

In [83]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings
import os

embedding = OpenAIEmbeddings()

db_openai = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_openai",
    embedding=embedding
)

In [84]:
# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index="test_index_openai"):
    db_openai.client.delete_by_query(index="test_index_openai", body={"query": {"match_all": {}}})
db_openai.add_documents(docs)
db_openai.client.indices.refresh(index="test_index_openai")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [85]:
query = "異常検知について知りたい"
results = db_openai.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_openai_embeddings", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}


[None, None, None, None]

In [31]:
query = "機械学習の機能ついて知りたい"
results = db_openai.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_openai_embeddings", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}


[None, None, None, None]

## 3-C. (HuggingFaceにあるE5モデルを利用） Elasticsearchのセマンティック検索

#### インジェスト

In [32]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

MODEL_ID="intfloat/multilingual-e5-base"

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_huggingface_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_huggingface_e5",
    embedding=embedding
)


In [33]:
db_huggingface_e5.client.delete_by_query(index="test_index_huggingface_e5", body={"query": {"match_all": {}}})
db_huggingface_e5.add_documents(docs)
db_huggingface_e5.client.indices.refresh(index="test_index_huggingface_e5")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

#### サーチ

In [34]:
query = "異常検知について知りたい"
results = db_huggingface_e5.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_huggingface_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}


[None, None, None, None]

In [41]:
query = "機械学習の機能ついて知りたい"
results = db_huggingface_e5.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_huggingface_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}


[None, None, None, None]

## 3-D. (ElasticsearchにアップしたE5モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [17]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_esml_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_esml_e5",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

In [19]:
if db_esml_e5.client.indices.exists(index="test_index_esml_e5"):
    db_esml_e5.client.delete_by_query(index="test_index_esml_e5", body={"query": {"match_all": {}}})
print(len(docs))
db_esml_e5.add_documents(docs)
db_esml_e5.client.indices.refresh(index="test_index_esml_e5")

6


ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [20]:
query = "異常検知について知りたい"
results = db_esml_e5.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_elastic_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}


[None, None, None, None]

In [23]:
query = "機械学習の機能ついて知りたい"
results = db_esml_e5.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_elastic_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}


[None, None, None, None]

## 3-E. Elasticsearch RRF ハイブリッド検索

### インジェスト

In [ ]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_esml_e5_hybrid = ElasticsearchStore(
    es_connection=es.options(request_timeout=3600),
    index_name="test_index_esml_e5_hybrid",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

In [ ]:
if db_esml_e5_hybrid.client.indices.exists(index="test_index_esml_e5_hybrid"):
    db_esml_e5_hybrid.client.delete_by_query(index="test_index_esml_e5_hybrid", body={"query": {"match_all": {}}})
print(len(docs))
db_esml_e5_hybrid.add_documents(docs)
db_esml_e5_hybrid.client.indices.refresh(index="test_index_esml_e5_hybrid")

6


ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [ ]:
query = "異常検知について知りたい"
results = db_esml_e5_hybrid.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "hybrid_elastic_bm25_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}


[None, None, None, None]

In [ ]:
query = "機械学習の機能ついて知りたい"
results = db_esml_e5_hybrid.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "hybrid_elastic_bm25_e5", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}


[None, None, None, None]

## 3-F. (HuggingFaceにある英語用モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [36]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

huggingface_api_Key = getpass("Enter your HF Inference API Key:\n\n")
MODEL_ID="sentence-transformers/all-distilroberta-v1"

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_huggingface_distilroberta = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_huggingface_distilroberta",
    embedding=embedding
)


In [37]:
db_huggingface_distilroberta.client.delete_by_query(index="test_index_huggingface_distilroberta", body={"query": {"match_all": {}}})
db_huggingface_distilroberta.add_documents(docs)
db_huggingface_distilroberta.client.indices.refresh(index="test_index_huggingface_distilroberta")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

### サーチ

In [38]:
query = "異常検知について知りたい"
results = db_huggingface_distilroberta.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_distilroberta", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}


[None, None, None, None]

In [40]:
query = "機械学習の機能ついて知りたい"
results = db_huggingface_distilroberta.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "vector_distilroberta", "query": query, "result": [element.metadata for element in results]})

{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/siem', 'title': 'SIEM（セキュリティ情報およびイベント管理）とは？ | Elastic SIEM', 'description': 'セキュリティ情報およびイベント管理（SIEM）を使って一歩先を行くサイバー脅威対策を行いましょう。SIEMの重要性、さまざまなメリット、ユースケースのほか、組織のデータと資産のセキュリティを確保するためのベストプラクティスを説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}


[None, None, None, None]

## 3-G.（Elastic ELSERを利用） ElasticsearchcでSparse Vector検索

#### インジェスト

In [179]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

db_elser = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
)

In [182]:
if es.indices.exists(index="test_index_elser"):
    db_elser.client.delete_by_query(index="test_index_elser", body={"query": {"match_all": {}}})
pprint(docs)
db_elser.add_documents(docs)
db_elser.client.indices.refresh(index="test_index_elser")

[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニアリングのスピードを高め、イノベーション

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

#### サーチ

In [166]:
query = "異常検知について知りたい"
results = db_elser.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "sparsevector_elastic_elser", "query": query, "result": [element.metadata for element in results]})

{'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic'}
{'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic'}
{'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic'}
{'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic'}


[None, None, None, None]

In [171]:
query = "機械学習の機能ついて知りたい"
results = db_elser.similarity_search(query)
[print(element.metadata) for element in results]
result_list.append({"search_logic": "sparsevector_elastic_elser", "query": query, "result": [element.metadata for element in results]})

{'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic'}
{'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic'}
{'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic'}
{'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp', 'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic'}


[None, None, None, None]

# 4. RAG検索: LLMを組み合わせてみよう

## 4-1. RetrievalQA chainでQA回答をする

### 4-1-1. RetrievalQA chainのファンクションを作成

In [175]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def ask_llm(vectorstore, question, template=None):
    if not template:
        template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    result = qa_chain({"query": question})
    pprint("------------ result -----------")
    pprint(result["result"])
    pprint("------------ source_documents -----------")
    pprint(result['source_documents'])
    return {"query": question, "result": ["result"] + result['source_documents']}

### 4-1-2. 検索文書を文字数Splitする
様々なSplit方法を試してみます。

#### 4-1-2-A. RecursiveCharacterTextSplitterでSplitする

In [138]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("---- ALL DOCS----")
pprint(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10----")

for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

---- ALL DOCS----
[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニアリ

#### 4-1-2-B. NLTKTextSplitterでSplitする

In [192]:
%pip install -q nltk
from langchain.text_splitter import NLTKTextSplitter

print("---- ALL DOCS----")
pprint(docs)
text_splitter = NLTKTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10 ----")
[pprint(split.metadata['source'] + "\n" + split.page_content) for split in all_splits[:10]]

Note: you may need to restart the kernel to use updated packages.
---- ALL DOCS----
[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：

[None, None, None, None, None, None]

#### 4-1-2-C. CharacterTextSplitterでSplitする

In [366]:
from langchain.text_splitter import CharacterTextSplitter

print("---- All DOCS ----")
pprint(docs)
text_splitter = CharacterTextSplitter(separator = "\n\n", chunk_size = 500, chunk_overlap = 0, is_separator_regex=True)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10 ----")
for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

Created a chunk of size 855, which is longer than the specified 500
Created a chunk of size 698, which is longer than the specified 500
Created a chunk of size 1011, which is longer than the specified 500
Created a chunk of size 642, which is longer than the specified 500
Created a chunk of size 973, which is longer than the specified 500
Created a chunk of size 642, which is longer than the specified 500


---- All DOCS ----
[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニア

#### 4-1-2-D. HTMLHeaderTextSplitterでSplitする (うまくいかない)

In [297]:
%pip -q install lxml
%pip -q install markdownify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [318]:
from langchain.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from pprint import pprint

def extract_main_content(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    main_html = soup.find("main")
    scripts = main_html.find_all('script')
    for s in scripts:
        s.extract()

    if main_html:
        return main_html.prettify()
    return None

# https://www.elastic.co/jp/sitemapから取得
urls = [
    "https://www.elastic.co/jp/what-is/root-cause-analysis",
    "https://www.elastic.co/jp/what-is/anomaly-detection",
    "https://www.elastic.co/jp/what-is/siem",
    "https://www.elastic.co/jp/what-is/opentelemetry",
    "https://www.elastic.co/jp/what-is/aiops",
    "https://www.elastic.co/jp/what-is/kubernetes-monitoring"
    ]
loader = AsyncHtmlLoader(urls) # Use to load raw html into page_content
html_docs = loader.load()

for html_doc in html_docs:
    pprint(html_doc.metadata)
    print("======= 抽出前 =======")
    pprint(html_doc.page_content[:1000])
    print("======= 抽出後 =======")
    html_doc.page_content = extract_main_content(html_doc.page_content)
    pprint(html_doc.page_content[:1000])
    print("==============")

Fetching pages: 100%|##########| 6/6 [00:00<00:00,  9.00it/s]

{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
======= 抽出前 =======
('<!DOCTYPE html><html lang="ja-jp"><head><meta '
 'charSet="utf-8"/><title>根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | '
 'Elastic</title><link rel="apple-touch-icon" sizes="57x57" '
 'href="/apple-icon-57x57.png"/><link rel="apple-touch-icon" sizes="60x60" '
 'href="/apple-icon-60x60.png"/><link rel="apple-touch-icon" sizes="72x72" '
 'href="/apple-icon-72x72.png"/><link rel="apple-touch-icon" sizes="76x76" '
 'href="/apple-icon-76x76.png"/><link rel="apple-touch-icon" sizes="114x114" '
 'href="/apple-icon-114x114.png"/><link rel="apple-touch-icon" sizes="120x120" '
 'href="/apple-icon-120x120.pn/g"/><link rel="apple-touch-icon" '
 'sizes="144x144" href="/apple-icon-144x144.png"/><link rel="apple-touch-icon" '
 'sizes="152x152" href="/apple-icon-152x152.png"/><link rel="apple-touch-icon" '
 'sizes="180x180" href="/apple-icon-180x180.png"/><link rel="apple-touch-icon" '
 'sizes="192x192" href="/apple-icon-192

In [319]:
from langchain.text_splitter import HTMLHeaderTextSplitter

headers_to_split_on = [
    # ("h1", "Header 1"),
    # ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]
all_html_splits = []
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on, return_each_element=False)
for html_doc in html_docs:
    html = f"<html><body>{html_doc.page_content}</body></html>"
    # pprint(BeautifulSoup(html, "html.parser").find_all("h3"))
    html_header_splits_of_1doc = html_splitter.split_text(html)
    print("---- HTML SPLITTED first 10 ----")
    for split in html_header_splits_of_1doc:
        pprint(split)
        # pprint(BeautifulSoup(split.page_content, "html.parser").find_all("div"))
        # if split.metadata.get("Header 3"):
        #     split.metadata['source'] = html_doc.metadata['source']
        #     pprint(split.metadata)
        #     # pprint(split.page_content[:100])
        #     # print("")
        #     all_html_splits.append(split)

---- HTML SPLITTED first 10 ----
Document(page_content='ソフトウェア開発における根本原因分析（RCA）とは何か？  \nElasticオブザーバビリティで、根本原因分析をすばやく実施  \n根本原因分析とは  \n根本原因分析（RCA）は、対症療法を試すのではなく、ソフトウェア開発チームが問題の核心から問題を特定して解決するために使用する、実績のあるトラブルシューティング手法です。 根本原因分析とは、構造化された段階的なプロセスであり、関連データを収集、分析し、それに対処する解決策をテストすることによって、主要な根本原因を探し出すように設計されています。  \n根本原因分析が重要である理由  \n根本原因分析がソフトウェア開発において不可欠なのは、体系的なアプローチによって、チームがより効率的にトラブルシューティングを行い、問題の再発を防ぐ長期的な解決策を開発できるからです。エラーや不具合の根本原因に対処することで、開発者はシステムの安定性、信頼性、効率性を確保し、コストのかかるダウンタイムを減らし、開発プロセスをスピードアップすることができます。また、RCAは開発者が問題の影響度と重大性に基づいて優先度を設定し、最も重大な問題に最初に取り組めるようにします。  \n根本原因分析を実行する方法  \n科学や工学から製造や医療に至るまで、あらゆる業種や分野で問題解決の手法として応用されている根本原因分析では、システムの欠陥や障害の根本的な要因を特定、理解するために、特定の一連のステップを実行する必要があります。ソフトウェア開発における根本原因分析の実施に関わるステップは、同じ普遍的なRCAの原則に従います。  \n手順1：問題を定義し、アラートを設定する（可能な場合） RCAの最初のステップは、問題を定義し、それを明確に理解することです。これには、アプリケーションの異常動作、システムパフォーマンスの低下、セキュリティインシデントなどの潜在的な問題を監視するためのアラートの設定が含まれます。 手順2．データを収集して分析し、潜在的な原因を判定する 問題が定義された後は、次のステップはデータの収集と分析です。これには、システムログ、アプリケーションパフォーマンスメトリック、ユーザーフィードバック、その他の関連するデー

In [262]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

for split in all_html_splits[2:3]:
    pprint(split.metadata)
    pprint(split.page_content)

# Split
all_splits = text_splitter.split_documents(all_html_splits)
print("---- SPLITTED first 10 ----")
for split in all_splits:
    pprint(split.metadata)
    pprint(split.page_content)



{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
('Search  \n'
 'Platform  \n'
 'Elasticsearch Platform  \n'
 'オブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。  \n'
 'Elasticの概要  \n'
 'ELK Stack  \n'
 'データインジェストから検索、分析、可視化まで、すべてが思いのまま  \n'
 'KibanaElasticsearch統合機能群  \n'
 'ELK Stackの概要  \n'
 'Elastic Cloud  \n'
 'お好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。  \n'
 'Cloudの概要  \n'
 'パートナーセールスへのお問い合わせ  \n'
 'ソリューション  \n'
 'オブザーバビリティ  \n'
 'アプリやインフラをまとめて可視化し、問題を先回りして修正できます。  \n'
 'ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetry  \n'
 'オブザーバビリティの概要  \n'
 'セキュリティ  \n'
 'サイバー脅威を広範囲で防御、調査し迅速に対応できます。  \n'
 '継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化  \n'
 'セキュリティの概要  \n'
 'Search  \n'
 'あらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。  \n'
 '生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポート  \n'
 'Searchの概要  \n'
 '業界別  \n'
 '公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車  \n'
 'すべての業界を表示  \n'
 '導入事例  \n'
 'ソリューション別  \n'
 '次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。  \n'
 'オブザーバビリティ

## 4-2. Retrieverを作成して、QAする

### 4-2-A. OpenAI Embedding

#### ベクトルストアの作成

In [139]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings
import os

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

embedding = OpenAIEmbeddings()

pprint(es.info())
db_split_openai = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_openai",
    embedding=embedding
)

ObjectApiResponse({'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


#### インジェスト

In [140]:
if es.indices.exists(index="test_split_index_openai"):
    db_split_openai.client.delete_by_query(index="test_split_index_openai", body={"query": {"match_all": {}}})
db_split_openai.add_documents(all_splits)
db_split_openai.client.indices.refresh(index="test_split_index_openai")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

#### セマンティック検索

In [141]:
query = "異常検知について知りたい"
results = db_split_openai.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}


[None, None, None, None]

#### RAG検索

In [146]:
query = "異常検知について知りたい"
result = ask_llm(vectorstore=db_split_openai, question=query)
result_list.append({"search_logic": "rag_openai_embedding", "query": query, "result": result})


'------------ result -----------'
('異常検知は、通常の状態から逸脱しているデータセットやシステムのデータポイントを特定するプロセスです。異常検知は、潜在的な問題や脅威を素早く絞り込み、システムの整合性と信頼性を維持管理するために有用です。異常検知は、さまざまな領域において多様なユースケースがあります。 '
 'Thanks for asking!')
'------------ source_documents -----------'
[Document(page_content='ログイン無料トライアルを始めるセールスへのお問い合わせ異常検知とは何ですか？オブザーバビリティソリューションについて見てみましょう異常検知の定義異常検知とは、通常の状態を逸脱しているデータセットやシステムにおけるデータポイントを特定するプロセスです。データ解析中や機械学習の異常検知では、ほとんどのデータ内の通常のパターンや統計モデルに一致しないインスタンスにフラグを立てます。異常は外れ値、想定外の変化、またはエラーとして出現し、分析しているデータの種類や事前に設定したパラメータによって異なります。異常検知が有用なのは、潜在的な問題や脅威を素早く効率的に絞り込み、システムの整合性と信頼性を維持管理できる点です。異常の種類異常検知システムが見つけられる異常のいくつかの種類を以下に示します。（こういったカテゴリは互いに排他的するものではなく、異常は複数のカテゴリの特性を同時に示す場合があるという点に注意することが重要です。）ポイント異常（Point', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}),
 Document(page_content='のアプローチ、信号処理技法）も異常検知に活用できるということは言及しておく価値があり

In [132]:
query = "機械学習の機能ついて知りたい"
result = ask_llm(vectorstore=db_split_openai, question=query)
result_list.append({"search_logic": "rag_openai_embedding", "query": query, "result": result})

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/2322012183.py", line 1, in <module>
    ask_llm(vectorstore=db_split_openai, question="機械学習の機能ついて知りたい")
  File "/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/2400514937.py", line 23, in ask_llm
    result = qa_chain({"query": question})
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 308, in __call__
    except BaseException as e:
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 302, in __call__
    try:
  File "/Users/nobuhikosekiya/lab/projects/elasticsearc

In [133]:
query = "機械学習の機能の名前を知りたい"
result = ask_llm(vectorstore=db_split_openai, question=query)
result_list.append({"search_logic": "rag_openai_embedding", "query": query, "result": result})

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/3539541129.py", line 1, in <module>
    ask_llm(vectorstore=db_split_openai, question="機械学習の機能の名前を知りたい")
  File "/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/2400514937.py", line 23, in ask_llm
    result = qa_chain({"query": question})
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 308, in __call__
    except BaseException as e:
  File "/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 302, in __call__
    try:
  File "/Users/nobuhikosekiya/lab/projects/elasticsear

### 4-2-B. HuggingFace API: E5モデル

In [404]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

if not huggingface_api_Key:
    huggingface_api_Key = getpass("Enter your HF Inference API Key:\n\n")
MODEL_ID="intfloat/multilingual-e5-base"

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_split_huggingface_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_huggingface_e5",
    embedding=embedding
)

In [405]:
if es.indices.exists(index="test_split_index_huggingface_e5"):
    db_split_huggingface_e5.client.delete_by_query(index="test_split_index_huggingface_e5", body={"query": {"match_all": {}}})
db_split_huggingface_e5.add_documents(all_splits)
db_split_huggingface_e5.client.indices.refresh(index="test_split_index_huggingface_e5")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [406]:
query = "異常検知について知りたい"
results = db_split_huggingface_e5.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}


[None, None, None, None]

In [407]:
query = "機械学習の機能ついて知りたい"
results = db_split_huggingface_e5.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}


[None, None, None, None]

In [102]:
query = "異常検知について知りたい"
result = ask_llm(vectorstore=db_split_huggingface_e5, question=query)
result_list.append({"search_logic": "rag_huggingface_e5", "query": query, "result": result})


NameError: name 'db_split_huggingface_e5' is not defined

In [409]:
query = "機械学習の機能ついて知りたい"
result = ask_llm(vectorstore=db_split_huggingface_e5, question=query)
result_list.append({"search_logic": "rag_huggingface_e5", "query": query, "result": result})

('機械学習の機能には、異常検知、根本原因分析、セキュリティ関連データの分析、自動修正機能、継続的な改善が含まれます。これらの機能を使用することで、システムの問題を特定し、解決することができます。Thanks '
 'for asking!')
[Document(page_content='送を行います。言語固有のOpenTelemetry', metadata={'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}),
 Document(page_content='ることができます。システムの最適な性能からの逸脱を監視して、非効率性を見分けることができます。これは、使用しているプロセスの合理化と、システムの全体的な改善のためのアイディアの考案に役立てることができます。サービスの混乱であれ、問題を指摘する顧客サービスの問い合わせの異様な増加であれ、異常検知によって顧客満足度に影響を与える異常を見つけられるため、異常検知は顧客サービスにとって極めて重要です。機械学習ベースの異常検知は、使用しているシステムを常に監視できるという追加のメリットがあります。規制要件や業界標準からの逸脱を見分けることすらできるため、規制遵守にも役立ちます。異常検知の制限と課題異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。管理型機械学習モデルは、正常に作動させるために多くのラベル付きデータを必要とします。異常検知訓練モデルでラベル付きデータの量が不十分だった場合、その正確性に問題が出ます。異常検知を測定するアルゴリズムに不正確なしきい値を設定すると、さまざまエラー報告の原因となり得ます。アルゴリズムを今では古くなった過去のモデル', metadata={'source': 'https://www.elastic.co/jp/w

In [410]:
query = "機械学習の機能の名前を知りたい"
result = ask_llm(vectorstore=db_split_huggingface_e5, question=query)
result_list.append({"search_logic": "rag_huggingface_e5", "query": query, "result": result})

'自動修正機能、管理型ML異常検知技法、非管理型ML異常検知技法、半管理型ML異常検知技法、統計モデル、ルールベース、OpenTelemetry'
[Document(page_content='で、さらに一歩踏み込むことができます。自動修正機能を使うと、エンドユーザーが問題の発生を認識する前に、問題を解決できます。継続的に改善される：機械学習を活用する場合は、時間の経過に伴い能力が改善されていきます。問題を特定し、解決することで、モデルは学習、適応することができ、将来の問題に効率的に対処できるようになります。AIOpsの能力', metadata={'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}),
 Document(page_content='ープに分かれます。どの技法を選ぶかは、解決しようとしている問題に固有の要件と、ラベル付けしたデータがどれ程あるかによって異なります。管理型ML異常検知技法は、訓練中に通常と異常の両方のインスタンスを明確に定義する、ラベル付けしたデータを必要とする技法です。このモデルは通常データのパターンを学習し、次に学習内容に基づいて、新しいデータを通常か異常のいずれかに分類します。非管理型ML異常検知技法では、ラベル付けしたデータを使用しません。ここでは、異常はまれにしか起こらず、ほとんどのデータとは大きく異なっていると仮定しています。この技法は、通常ではないパターン、外れ値、通常行動からの逸脱を特定することを目的としています。半管理型ML異常検知技法では、ラベル付けしたデータとしていないデータを組み合わせて使用します。ラベル付けしたデータを活用して、通常行動のベースラインを確立し、次にラベル付けしていないデータを使って、このベースラインからの逸脱を特定します。これは特に、非構造化データで作業する場合に有用です。機械学習は異常検知で一般的に使用されますが、その他の技法（例えば統計モデル、ルールベース', me

### 4-2-C. Elastic MLのE5モデル

In [157]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID,
    input_field="text_field"
)

db_split_esml_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_esml_e5",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

db_split_esml_e5_hybrid = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_esml_e5_hybrid",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

In [158]:
if db_split_esml_e5.client.indices.exists(index="test_split_index_esml_e5"):
    db_split_esml_e5.client.delete_by_query(index="test_split_index_esml_e5", body={"query": {"match_all": {}}})

pprint(all_splits[:1])
for split in all_splits:
    db_split_esml_e5.add_documents([split])
    print(".", end='')
    db_split_esml_e5.client.indices.refresh(index="test_split_index_esml_e5")

[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'})]
.....................................................................................................................................................

In [29]:
if db_split_esml_e5_hybrid.client.indices.exists(index="test_split_index_esml_e5_hybrid"):
    db_split_esml_e5_hybrid.client.delete_by_query(index="test_split_index_esml_e5_hybrid", body={"query": {"match_all": {}}})

pprint(all_splits[:1])
for split in all_splits:
    db_split_esml_e5_hybrid.add_documents([split])
    print(".", end='')
    db_split_esml_e5_hybrid.client.indices.refresh(index="test_split_index_esml_e5_hybrid")

[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'})]
.....................................................................................................................................................

In [34]:
query = "異常検知について知りたい"
results = db_split_esml_e5.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}


[None, None, None, None]

In [35]:
query = "異常検知について知りたい"
results = db_split_esml_e5_hybrid.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}


[None, None, None, None]

In [37]:
query = "機械学習の機能ついて知りたい"
results = db_split_esml_e5.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}


[None, None, None, None]

In [38]:
query = "機械学習の機能ついて知りたい"
results = db_split_esml_e5_hybrid.similarity_search(query)
[print(element.metadata) for element in results]

{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}


[None, None, None, None]

In [45]:
query = "異常検知について知りたい"
result = ask_llm(vectorstore=db_split_esml_e5, question=query)
result_list.append({"search_logic": "rag_elastic_e5", "query": query, "result": result})

'------------ result -----------'
('異常検知は、通常の状態から逸脱しているデータやシステムの異常を特定するプロセスです。異常検知は、問題の早期検出やセキュリティの向上に役立ちます。異常検知は、データ解析や機械学習を使用して行われ、異常なパターンや統計モデルに一致しないインスタンスを検出します。異常検知は、サイバーセキュリティ、アプリケーション監視、不正検知など、さまざまな領域で利用されます。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='その他の疑わしい活動を見分けることによって行います。ハードウェアの維持管理：異常検知を活用して、使用しているハードウェアの動作を監視することができます。これにはCPU温度、ファン速度、電圧レベルといった要因が含まれます。差し迫った不具合の兆候が見えていることを意味する異常にフラグを立てられるので、機能停止する前に修理を開始できます。ユーザー行動分析：異常検知は、ユーザーのプロファイリングを回避しながら、アプリケーション、ウェブサイト、その他のオンラインプラットフォームでのユーザー行動パターンと傾向を分析できます。これは、通常とは異なるユーザーの操作の特定と、セキュリティ対策の個人化に役立ちます。異常検知の利点異常検知には多くの利点があります。あなたの組織に役立つ例をいくつか紹介します。これで問題が大きく膨らむ前に、問題の早期検出ができます。早期に異常を検知すると、行動を起こして損傷や混乱を阻止するための処置を講じることができます。異常検知はサイバー攻撃や詐欺行為といった、悪意のある疑わしい行動の特定において、重要な役割を果たします。また脅威の可能性を素早く見分けて、セキュリティを改善す', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を

In [46]:
query = "機械学習の機能ついて知りたい"
result = ask_llm(vectorstore=db_split_esml_e5, question=query)
result_list.append({"search_logic": "rag_elastic_e5", "query": query, "result": result})

'------------ result -----------'
('機械学習を活用することで、エンドユーザーが問題を解決する前に自動的に修正することができます。また、機械学習は時間の経過とともに能力が改善され、将来の問題にも効率的に対処できるようになります。AIOpsの能力によって、問題の特定と解決が行われ、モデルが学習と適応を行うことができます。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='送を行います。言語固有のOpenTelemetry', metadata={'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}),
 Document(page_content='で、さらに一歩踏み込むことができます。自動修正機能を使うと、エンドユーザーが問題の発生を認識する前に、問題を解決できます。継続的に改善される：機械学習を活用する場合は、時間の経過に伴い能力が改善されていきます。問題を特定し、解決することで、モデルは学習、適応することができ、将来の問題に効率的に対処できるようになります。AIOpsの能力', metadata={'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}),
 Document(page_content='Whys「5', metadata={'source': 'h

In [48]:
query = "機械学習の機能の名前を知りたい"
result = ask_llm(vectorstore=db_split_esml_e5, question=query)
result_list.append({"search_logic": "rag_elastic_e5", "query": query, "result": result})

'------------ result -----------'
"I don't know the name of the machine learning feature. Thanks for asking!"
'------------ source_documents -----------'
[Document(page_content='送を行います。言語固有のOpenTelemetry', metadata={'source': 'https://www.elastic.co/jp/what-is/opentelemetry', 'title': 'OpenTelemetryとは？ | Elastic', 'description': 'OpenTelemetryとは何か、どのように機能するのか、その用途と利点について詳細をご覧ください。オープンテレメトリAPIやSDKなど、さまざまなインストルメンテーション方法をレビューします。', 'language': 'ja-jp'}),
 Document(page_content='Whys「5', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}),
 Document(page_content='で、さらに一歩踏み込むことができます。自動修正機能を使うと、エンドユーザーが問題の発生を認識する前に、問題を解決できます。継続的に改善される：機械学習を活用する場合は、時間の経過に伴い能力が改善されていきます。問題を特定し、解決することで、モデルは学習、適応することができ、将来の問題に効率的に対処できるようになります。AIOpsの能力', metadata={'source': 'https://www.elastic.co/jp/what-is/aiops', 'title':

### 4-2-D. Elastic ELSER

In [185]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore

db_split_elser = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
)

In [320]:
%pip install -q numpy
import numpy as np
# Define the size of sublists
chunk_size = 10

# Using numpy.array_split to split the list
split_list = np.array_split(all_splits, len(all_splits) // chunk_size)

if es.indices.exists(index="test_split_index_elser"):
    db_split_elser.client.delete_by_query(index="test_split_index_elser", body={"query": {"match_all": {}}})

for splits in split_list:
    db_split_elser.add_documents(splits)
    db_split_elser.client.indices.refresh(index="test_split_index_elser")

Note: you may need to restart the kernel to use updated packages.


In [324]:
query = "異常検知について知りたい"
results = db_split_elser.similarity_search(query)
[print(element) for element in results]

page_content='異常検知の制限と課題  \n異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。' metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}
page_content='に、スケーラビリティが問題となることがあります。組織は、現在完全に処理する必要がある以上の、計算リソースが必要となるかもしれません。' metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}
page_content='とができます。レイテンシとエラーの相関レイテンシとエラー率に関連するデータを分析し、両者の相関関係を特定することで、エラーとパフォーマンス問題の間のパターンと関係を見つけ、根本原因を突き止めることができます。' metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
page_content='異常の種類  \n異常検知システムが見つけられる異常のいくつかの種類を以下に示します。（こういったカテゴリは互いに排他的するものではなく、異常は複数のカテゴリの特性を同時に示す場合があるという点に注意することが重要です。）' metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}


[None, None, None, None]

In [325]:
query = "機械学習の機能ついて知りたい"
results = db_split_elser.similarity_search(query)
[print(element) for element in results]

page_content='異常検知の制限と課題  \n異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。' metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}
page_content='機械学習とAIOps検索、可視化、そして機械学習は、異常を特定し、問題の根本原因を浮き彫りにするのに役立ちます。これにより、十分な情報に基づいた決断を下し、迅速に是正措置を講じることができます。分散トレーシング分散トレーシングで複雑な分散システムを通過するリクエストのフローを追跡、分析することで、コンポーネントやサービス間の連携についてのインサイトが得られ、問題を引き起こしている可能性のあるボトルネックやその他の問題を特定するのに役立ちます。ログパターン分析アプリケーションとインフラストラクチャーによって生成されたログのパターンとトレンドを分析し、問題の根本原因を特定するだけでなく、ソフトウェアのパフォーマンスに影響を与える可能性のある異常、エラー、その他の問題を検出します。サービス依存関係マッピングシステム内の異なるコンポーネント間の関係と依存関係を特定することで、問題を引き起こしている可能性のあるサービスの依存関係を自動的にマッピングし、あるコンポーネントの変更がシステムの残りの部分にどのような影響を与えるかを理解することができます。レイテンシとエラーの相関レイテンシとエラー率に' metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
page_content='コミュニティフォーラムダウンロード  \nつながる  \nテクノロジーやイノベーション、ニュースに関する最新情報をお届けします。  \nイベントブログ  \n学習  \nスキルを磨き、将来の成功につなげましょう。  \n開始するElasticのリソースコンサルティングサービストレーニングコース＆認定資格Security Labs  \nヘルプ  \nどのような問題でも、必要なサポートをご提供します。  \nお問い合わせサポートセンター  \nElasticの最新情報' metad

[None, None, None, None]

In [326]:
ask_llm(vectorstore=db_split_elser, question="異常検知について知りたい")

('異常検知は、異常なデータやパターンを検出する技術です。異常検知技法にはスケーラビリティの制限やデータの相関関係の特定などの課題があります。異常検知は異常の種類を特定するための有用な手法です。Thanks '
 'for asking!')
[Document(page_content='異常検知の制限と課題  \n異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}),
 Document(page_content='に、スケーラビリティが問題となることがあります。組織は、現在完全に処理する必要がある以上の、計算リソースが必要となるかもしれません。', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}),
 Document(page_content='とができます。レイテンシとエラーの相関レイテンシとエラー率に関連するデータを分析し、両者の相関関係を特定することで、エラーとパフォーマンス問題の間のパターンと関係を見つけ、根本原因を突き止めることができます。', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}),
 Document(page_content='異常の種類  \n異常検知システムが見つけられる異常のいくつかの種類を以下に示します。（こういったカテゴリは互いに排他的するものではなく、異常は複数のカテゴリの特性を同時に示す場合があるという点に注意することが重要です。）', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'})]


In [327]:
ask_llm(vectorstore=db_split_elser, question="機械学習の機能ついて知りたい")

'機械学習は異常を特定し、問題の根本原因を浮き彫りにするのに役立ちます。これにより、十分な情報に基づいた決断を下し、迅速に是正措置を講じることができます。'
[Document(page_content='異常検知の制限と課題  \n異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}),
 Document(page_content='機械学習とAIOps検索、可視化、そして機械学習は、異常を特定し、問題の根本原因を浮き彫りにするのに役立ちます。これにより、十分な情報に基づいた決断を下し、迅速に是正措置を講じることができます。分散トレーシング分散トレーシングで複雑な分散システムを通過するリクエストのフローを追跡、分析することで、コンポーネントやサービス間の連携についてのインサイトが得られ、問題を引き起こしている可能性のあるボトルネックやその他の問題を特定するのに役立ちます。ログパターン分析アプリケーションとインフラストラクチャーによって生成されたログのパターンとトレンドを分析し、問題の根本原因を特定するだけでなく、ソフトウェアのパフォーマンスに影響を与える可能性のある異常、エラー、その他の問題を検出します。サービス依存関係マッピングシステム内の異なるコンポーネント間の関係と依存関係を特定することで、問題を引き起こしている可能性のあるサービスの依存関係を自動的にマッピングし、あるコンポーネントの変更がシステムの残りの部分にどのような影響を与えるかを理解することができます。レイテンシとエラーの相関レイテンシとエラー率に', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}),
 Document(page_content='コミュニティフォーラムダウンロード  \nつながる  \nテクノロジーやイノベーション、ニュースに関する最新情報をお届けします。  \nイベントブログ  \n学習  \nスキルを磨き、将来の成功につなげましょう。  \n開始するElasticのリソースコ

In [328]:
ask_llm(vectorstore=db_split_elser, question="機械学習の機能の名前を知りたい")

('機械学習の機能の名前は、異常検知、分散トレーシング、ログパターン分析、サービス依存関係マッピング、レイテンシとエラーの相関です。 \n'
 'Thanks for asking!')
[Document(page_content='機械学習とAIOps検索、可視化、そして機械学習は、異常を特定し、問題の根本原因を浮き彫りにするのに役立ちます。これにより、十分な情報に基づいた決断を下し、迅速に是正措置を講じることができます。分散トレーシング分散トレーシングで複雑な分散システムを通過するリクエストのフローを追跡、分析することで、コンポーネントやサービス間の連携についてのインサイトが得られ、問題を引き起こしている可能性のあるボトルネックやその他の問題を特定するのに役立ちます。ログパターン分析アプリケーションとインフラストラクチャーによって生成されたログのパターンとトレンドを分析し、問題の根本原因を特定するだけでなく、ソフトウェアのパフォーマンスに影響を与える可能性のある異常、エラー、その他の問題を検出します。サービス依存関係マッピングシステム内の異なるコンポーネント間の関係と依存関係を特定することで、問題を引き起こしている可能性のあるサービスの依存関係を自動的にマッピングし、あるコンポーネントの変更がシステムの残りの部分にどのような影響を与えるかを理解することができます。レイテンシとエラーの相関レイテンシとエラー率に', metadata={'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}),
 Document(page_content='異常検知の制限と課題  \n異常検知技法には、ある種の制限と課題があります。以下に、知っておくべき欠点をいくつか挙げます。', metadata={'source': 'https://www.elastic.co/jp/what-is/anomaly-detection'}),
 Document(page_content='管理型機械学習モデルは、正常に作動させるために多くのラベル付きデータを必要とします。異常検知訓練モデルでラベル付きデータの量が不十分だった場合、その正確性に問題が出ます。異常検知を測定するアルゴリズムに不正確なしきい値を設定

## 4-3. Self Query Retrieverで、質問文から検索のフィルターをかける

In [50]:
!python3 -m pip install -qU lark elasticsearch langchain openai

from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from getpass import getpass

In [51]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es.options(request_timeout=3600),
    model_id=MODEL_ID
)

db_esml_e5 = ElasticsearchStore(
    es_connection=es.options(request_timeout=3600),
    index_name="test_index_esml_e5",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

db_esml_e5_hybrid = ElasticsearchStore(
    es_connection=es.options(request_timeout=3600),
    index_name="test_index_esml_e5_hybrid",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

In [52]:
# Add details about metadata fields
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The url of the document",
        type="string or list[string]",
    )
]

document_content_description = "FAQ for elasticsearch"

# Set up openAI llm with sampling temperature 0
llm = OpenAI(temperature=0)

# instantiate retriever
retriever = SelfQueryRetriever.from_llm(
    llm, db_esml_e5, document_content_description, metadata_field_info, verbose=True
)

In [54]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.self_query").setLevel(logging.INFO)

In [59]:
results = retriever.get_relevant_documents("Kubernetesに関連した機械学習検知はありますか？")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='Kubernetes 機械学習 検知' filter=None limit=None


{'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/anomaly-detection', 'title': '異常検出とは何ですか？|包括的な異常検出のガイド | Elastic', 'description': '異常検出を定義し、それがどのように作動するのかを理解し、データセットで見つかる一般的な種類の異常検知を調査します。さまざまな異常検知技法について学びます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis', 'title': '根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | Elastic', 'description': '根本原因分析（RCA）を定義し、根本原因分析の実施方法や開発者向けのRCAツールなど、効果的なソフトウェア開発におけるRCAの役割を理解できます。', 'language': 'ja-jp'}
{'source': 'https://www.elastic.co/jp/what-is/aiops', 'title': 'AIOps | Elastic', 'description': 'IT運用のための人工知能）AIOpsソリューションは、人工知能と機械学習を活用して、IT運用におけるプロセスの自動化を支援します。\r\n', 'language': 'ja-jp'}


[None, None, None, None]

In [61]:
results = retriever.get_relevant_documents("urlにkubernetesが含まれる文書からKubernetesに関連した機械学習検知を教えてください？")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='machine learning detection related to Kubernetes' filter=Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='source', value='kubernetes') limit=None


{'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}


[None]

## 4-4. Agentsを使い、外部プログラムとしてElasticsearchのデータを検索させる

In [93]:
# Import things that are needed generically
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [94]:
llm = ChatOpenAI(temperature=0)

In [97]:
from langchain.tools import tool

@tool
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return f"Results for query {query}"

In [112]:
from langchain.tools import tool


@tool
def elasticsearch_api(query: str) -> str:
    """Searches the API for the query."""
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    fields = ["metadata", "page_content"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields)

    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        page_content = hit['_source']['page_content']
        score = hit['_score']
        print(f"_score: {score}, metadata: {metadata}")
        return page_content


In [152]:
from langchain.tools import tool

@tool
def askllm(query: str) -> str:
    """Searches the API for the query."""
    return ask_llm(vectorstore=db_split_openai, question=query)

In [159]:
# Load the tool configs that are needed.
tools = [
    Tool.from_function(
        func=askllm.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
    ),
    Tool.from_function(
        func=elasticsearch_api.run,
        name="Elasticsearch",
        description="Useful to get the whole document as-is",
    )
    ]

In [160]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [170]:
agent.run(
    "機械学習の機能の名前が記載されている文書をそのままの状態で表示してください"
)



> Entering new AgentExecutor chain...
I need to find a document that contains the names of machine learning functions and display it as-is.
Action: Elasticsearch
Action Input: "機械学習の機能の名前"_score: 2.1831076, metadata: {'source': 'https://www.elastic.co/jp/what-is/kubernetes-monitoring', 'title': 'Kubernetes監視とは？ | Elastic', 'description': 'このガイドでは、Kubernetes監視を定義し、Kubernetes監視のベストプラクティスや課題といったさまざまなKubernetes監視の方法について説明します。', 'language': 'ja-jp'}

Observation: Kubernetes監視とは？ | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リア

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 9311 tokens. Please reduce the length of the messages.

# OpenAI Function Callの利用

In [173]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

# Load the tool configs that are needed.
tools = [
    Tool.from_function(
        func=askllm.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
    ),
    Tool.from_function(
        func=elasticsearch_api.run,
        name="MySearch",
        description="Useful to get the whole document as-is",
    )
    ]


from langchain.tools import MoveFileTool, format_tool_to_openai_function
functions = [format_tool_to_openai_function(t) for t in tools]

message = model.predict_messages(
    [HumanMessage(content="機械学習の機能の名前が記載されている文書をそのままの状態で表示してください")], functions=functions
)

message


AIMessage(content='', additional_kwargs={'function_call': {'name': 'MySearch', 'arguments': '{\n  "__arg1": "機械学習"\n}'}})

In [ ]:
import openai

def chat_gpt_with_function(text):
    # AIが呼び出せる関数の定義
    functions = [
        # 何をする関数かについて記述
        {
            "name": "Elasticsearch",
            "description": "Useful to get the whole document as-is",
            "parameters": {
                "type": "object",
                "properties": {
                    # location引数についての情報を記述
                    "query": {
                        "type": "string",
                        "description": "The query to search the document",
                    },
                },
                "required": ["query"],
            },
        }
    ]

    # ユーザーの入力から、Functions Callingが必要かどうか判断する
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": text},
        ],
        functions=functions,
        function_call="auto",
    )

    # Function Callingが必要なければ、そのままAIの回答になる
    message = response["choices"][0]["message"]

    # Functions Callingが必要な場合は、messageに関数名と引数を格納されている
    if message.get("function_call"):

        # messageから実行する関数と引数を取得
        function_name = message["function_call"]["name"]
        arguments = json.loads(message["function_call"]["arguments"])

        # 関数を実行
        if function_name == "weather":
            function_response = weather_function(
                location=arguments.get("location"),
            )

        # 関数の実行結果を元にAIの回答を生成
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": text},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

        return "AIの回答: " + second_response.choices[0]["message"]["content"]
    else:
        return "AIの回答: " + message["content"]


# 結果まとめ